In [1]:
# analytics
import pandas as pd 
import numpy as np
import scipy.stats as stats
import statsmodels.formula.api as smf
#spatial 
import osmnx as ox
import geopandas as gpd
import contextily as cx
# plotting 
import df2img
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
#settings
import warnings

#set output to 3 digits
pd.set_option("display.precision", 3)
#suppress warnings 
warnings.filterwarnings('ignore')


In [2]:
# import data
path = '/Users/philip/Documents/ESE/ESE_thesis/flood_experience/data/export/clean_n.csv'
df_n = pd.read_csv(path)
df_n.columns

Index(['id', 'state', 'zipcode', 'geographic_division', 'census_region',
       'county', 'experience', 'supplies', 'insured', 'involved',
       'learned_routes', 'made_plan', 'made_safer', 'planned_neighbors',
       'practiced_drills', 'documents', 'rainy_day', 'alerts',
       'family_communication', 'none', 'dont_know', 'age', 'sex', 'education',
       'race', 'homeownership', 'income', 'rentmortgage', 'rurality',
       'hazard_weight', 'geometry', 'zip_count'],
      dtype='object')

In [3]:
path = '/Users/philip/Documents/ESE/ESE_thesis/flood_experience/data/export/clean_k.csv'
df_k = pd.read_csv(path)
df_k.columns

Index(['id', 'state', 'zipcode', 'geographic_division', 'census_region',
       'county', 'awareness', 'perception', 'experience', 'floodzone',
       'efficacy', 'supplies', 'insured', 'involved', 'learned_routes',
       'made_plan', 'made_safer', 'planned_neighbors', 'practiced_drills',
       'documents', 'rainy_day', 'alerts', 'family_communication', 'none',
       'dont_know', 'age', 'sex', 'education', 'race', 'homeownership',
       'income', 'rentmortgage', 'rurality', 'hazard_weight', 'geometry',
       'zip_count'],
      dtype='object')

#### We will go through these outcome variables one by one

structural adaptation: 
- made_safer

non-structural adaptation: 
- insurance
- learned_rountes
- supplies
- involved
- made_plan
- practiced_drills
- alerts
- family_communication

In [ ]:
def r_square(model):
    # McKelvay-Zavoina
    xb = model.predict(linear=True) #fitted latent value
    var_xb = np.var(xb,ddof=1) # variance of xb
    r2_mz = var_xb / (var_xb + 1) # McKelvay-Zavoina R_2
    # McFadden
    r2_mf = model.prsquared
    return r2_mz


In [5]:
def probit (functions, determinant, data):
    # create a dataframe to capture results 
    results = pd.DataFrame(columns=['effect', 'p', 'marginal_effect', 'pseudoR_2', 'LLRp','BIC']) 
    # iterate over functions and compute models and capture results   
    for var in functions:
        model = smf.probit(formula=var, data=data).fit(disp=0) # run the model
        #fill the dataframe columns 
        results.at[var,'effect'] = model.params[determinant]
        results.at[var,'p'] = model.pvalues[determinant]
        temporary = model.get_margeff().summary_frame() 
        results.at[var,'marginal_effect'] = temporary.at[determinant,'dy/dx']
        results.at[var,'pseudoR_2'] = r_square(model)
        results.at[var,'LLRp'] = model.llr_pvalue
        results.at[var,'BIC'] = model.bic
    return results

In [6]:
# duplicate function but with logit regression 
def logit(functions, determinant, data):
    results = pd.DataFrame(columns=['effect', 'p', 'marginal_effect', 'odds','pseudoR_2', 'LLRp', 'BIC']) #capture also odds ratio
    for var in functions:
        model = smf.logit(formula=var, data=data).fit(disp=0) #change here
        results.at[var,'effect'] = model.params[determinant]
        results.at[var,'p'] = model.pvalues[determinant]
        temporary = model.get_margeff().summary_frame() 
        results.at[var,'marginal_effect'] = temporary.at[determinant,'dy/dx']
        results.at[var, 'odds'] = np.exp(model.params[determinant]) # odds
        results.at[var,'pseudoR_2'] = r_square(model)
        results.at[var,'LLRp'] = model.llr_pvalue
        results.at[var,'BIC'] = model.bic
    return results

## What is the effect of experience on preparedness?

In [7]:
determinant = 'experience'
experience = [
    'made_safer ~ experience',
    'documents ~ experience',
    'insured ~ experience',
    'learned_routes ~ experience',
    'supplies ~ experience',
    'involved ~ experience',
    'made_plan ~ experience', 
    'practiced_drills ~ experience', 
    'alerts ~ experience', 
    'family_communication ~ experience'
]

In [8]:
probit(functions=experience, determinant=determinant, data=df_n)

,effect,p,marginal_effect,pseudoR_2,LLRp,BIC
made_safer ~ experience,0.465,0.001,0.121,0.041,0.001,453.145
documents ~ experience,0.48,0.001,0.121,0.044,0.001,440.529
insured ~ experience,0.602,0.0,0.169,0.067,0.0,483.963
learned_routes ~ experience,0.79,0.0,0.222,0.11,0.0,484.98
supplies ~ experience,0.925,0.0,0.249,0.145,0.0,465.788
involved ~ experience,0.849,0.0,0.125,0.125,0.0,270.94
made_plan ~ experience,0.611,0.0,0.173,0.069,0.0,488.408
practiced_drills ~ experience,0.613,0.0,0.088,0.069,0.0,264.515
alerts ~ experience,0.475,0.001,0.125,0.043,0.001,458.457
family_communication ~ experience,0.489,0.003,0.084,0.045,0.003,314.055


In [9]:
logit(functions=experience, determinant=determinant, data=df_n)

,effect,p,marginal_effect,odds,pseudoR_2,LLRp,BIC
made_safer ~ experience,0.81,0.001,0.119,2.248,0.115,0.001,453.145
documents ~ experience,0.842,0.001,0.119,2.321,0.123,0.001,440.529
insured ~ experience,1.025,0.0,0.165,2.788,0.172,0.0,483.963
learned_routes ~ experience,1.334,0.0,0.216,3.795,0.261,0.0,484.98
supplies ~ experience,1.568,0.0,0.242,4.796,0.327,0.0,465.788
involved ~ experience,1.654,0.0,0.126,5.225,0.351,0.0,270.94
made_plan ~ experience,1.038,0.0,0.169,2.825,0.176,0.0,488.408
practiced_drills ~ experience,1.21,0.0,0.087,3.354,0.225,0.0,264.515
alerts ~ experience,0.826,0.001,0.123,2.283,0.119,0.001,458.457
family_communication ~ experience,0.932,0.003,0.083,2.538,0.147,0.003,314.055


## What is the effect of risk perception on preparedness?

In [10]:
determinant = 'perception'
perception = [
    'made_safer ~ perception',
    'documents ~ perception',
    'insured ~ perception',
    'learned_routes ~ perception',
    'supplies ~ perception',
    'involved ~ perception',
    'made_plan ~ perception', 
    'practiced_drills ~ perception', 
    'alerts ~ perception', 
    'family_communication ~ perception'
]

In [11]:
probit(functions=perception, determinant=determinant, data=df_k)

,effect,p,marginal_effect,pseudoR_2,LLRp,BIC
made_safer ~ perception,0.688,0.0,0.178,0.105,0.0,369.528
documents ~ perception,0.803,0.0,0.206,0.137,0.0,366.077
insured ~ perception,1.13,0.0,0.307,0.239,0.0,384.042
learned_routes ~ perception,0.85,0.0,0.25,0.151,0.0,413.939
supplies ~ perception,0.96,0.0,0.268,0.185,0.0,394.057
involved ~ perception,1.308,0.0,0.205,0.297,0.0,231.809
made_plan ~ perception,0.775,0.0,0.223,0.129,0.0,406.276
practiced_drills ~ perception,1.012,0.0,0.145,0.202,0.0,216.374
alerts ~ perception,0.884,0.0,0.228,0.162,0.0,366.632
family_communication ~ perception,0.742,0.0,0.131,0.12,0.0,261.846


In [12]:
logit(functions=perception, determinant=determinant, data=df_k)

,effect,p,marginal_effect,odds,pseudoR_2,LLRp,BIC
made_safer ~ perception,1.243,0.0,0.184,3.465,0.276,0.0,369.528
documents ~ perception,1.459,0.0,0.215,4.301,0.344,0.0,366.077
insured ~ perception,2.027,0.0,0.325,7.591,0.503,0.0,384.042
learned_routes ~ perception,1.482,0.0,0.257,4.401,0.351,0.0,413.939
supplies ~ perception,1.704,0.0,0.279,5.498,0.417,0.0,394.057
involved ~ perception,2.837,0.0,0.241,17.061,0.665,0.0,231.809
made_plan ~ perception,1.359,0.0,0.229,3.894,0.313,0.0,406.276
practiced_drills ~ perception,2.177,0.0,0.163,8.824,0.539,0.0,216.374
alerts ~ perception,1.607,0.0,0.238,4.987,0.389,0.0,366.632
family_communication ~ perception,1.487,0.001,0.139,4.423,0.353,0.0,261.846


## What is the effect of flood risk on preparedness?

In [13]:
determinant = 'floodzone'
floodzone = [
    'made_safer ~ floodzone',
    'documents ~ floodzone',
    'insured ~ floodzone',
    'learned_routes ~ floodzone',
    'supplies ~ floodzone',
    'involved ~ floodzone',
    'made_plan ~ floodzone', 
    'practiced_drills ~ floodzone', 
    'alerts ~ floodzone', 
    'family_communication ~ floodzone'
]

In [14]:
probit(functions=floodzone, determinant=determinant, data=df_k)

,effect,p,marginal_effect,pseudoR_2,LLRp,BIC
made_safer ~ floodzone,0.463,0.002,0.123,0.045,0.003,379.618
documents ~ floodzone,0.446,0.003,0.12,0.042,0.003,383.066
insured ~ floodzone,0.673,0.0,0.198,0.09,0.0,414.225
learned_routes ~ floodzone,0.564,0.0,0.173,0.065,0.0,431.47
supplies ~ floodzone,0.652,0.0,0.191,0.085,0.0,413.272
involved ~ floodzone,0.711,0.0,0.118,0.1,0.0,248.571
made_plan ~ floodzone,0.434,0.003,0.131,0.04,0.003,424.289
practiced_drills ~ floodzone,0.801,0.0,0.116,0.123,0.0,219.031
alerts ~ floodzone,0.852,0.0,0.218,0.137,0.0,365.711
family_communication ~ floodzone,0.561,0.001,0.101,0.064,0.001,266.994


In [15]:
logit(functions=floodzone, determinant=determinant, data=df_k)

,effect,p,marginal_effect,odds,pseudoR_2,LLRp,BIC
made_safer ~ floodzone,0.806,0.002,0.122,2.239,0.124,0.003,379.618
documents ~ floodzone,0.776,0.003,0.119,2.173,0.116,0.003,383.066
insured ~ floodzone,1.136,0.0,0.195,3.115,0.22,0.0,414.225
learned_routes ~ floodzone,0.946,0.0,0.171,2.576,0.164,0.0,431.47
supplies ~ floodzone,1.102,0.0,0.188,3.01,0.21,0.0,413.272
involved ~ floodzone,1.37,0.0,0.119,3.936,0.291,0.0,248.571
made_plan ~ floodzone,0.735,0.003,0.129,2.085,0.106,0.003,424.289
practiced_drills ~ floodzone,1.59,0.0,0.118,4.902,0.356,0.0,219.031
alerts ~ floodzone,1.476,0.0,0.216,4.376,0.323,0.0,365.711
family_communication ~ floodzone,1.065,0.001,0.101,2.9,0.199,0.001,266.994


## What is the effect of awareness on preparedness?

In [16]:
determinant = 'awareness'
awareness = [
    'made_safer ~ awareness',
    'documents ~ awareness',
    'insured ~ awareness',
    'learned_routes ~ awareness',
    'supplies ~ awareness',
    'involved ~ awareness',
    'made_plan ~ awareness', 
    'practiced_drills ~ awareness', 
    'alerts ~ awareness', 
    'family_communication ~ awareness'
]

In [17]:
probit(functions=awareness, determinant=determinant, data=df_k)

,effect,p,marginal_effect,pseudoR_2,LLRp,BIC
made_safer ~ awareness,0.639,0.0,0.166,0.093,0.0,370.706
documents ~ awareness,0.698,0.0,0.181,0.109,0.0,370.165
insured ~ awareness,0.751,0.0,0.217,0.124,0.0,408.127
learned_routes ~ awareness,0.899,0.0,0.26,0.168,0.0,407.601
supplies ~ awareness,0.978,0.0,0.269,0.193,0.0,389.071
involved ~ awareness,0.781,0.0,0.129,0.132,0.0,246.611
made_plan ~ awareness,0.814,0.0,0.232,0.142,0.0,401.538
practiced_drills ~ awareness,0.728,0.0,0.107,0.117,0.0,223.022
alerts ~ awareness,0.817,0.0,0.211,0.143,0.0,368.135
family_communication ~ awareness,0.768,0.0,0.134,0.129,0.0,259.117


In [18]:
logit(functions=awareness, determinant=determinant, data=df_k)

,effect,p,marginal_effect,odds,pseudoR_2,LLRp,BIC
made_safer ~ awareness,1.139,0.0,0.169,3.122,0.245,0.0,370.706
documents ~ awareness,1.243,0.0,0.184,3.468,0.279,0.0,370.165
insured ~ awareness,1.297,0.0,0.22,3.66,0.296,0.0,408.127
learned_routes ~ awareness,1.55,0.0,0.263,4.711,0.376,0.0,407.601
supplies ~ awareness,1.709,0.0,0.275,5.526,0.422,0.0,389.071
involved ~ awareness,1.563,0.0,0.136,4.775,0.38,0.0,246.611
made_plan ~ awareness,1.412,0.0,0.235,4.105,0.333,0.0,401.538
practiced_drills ~ awareness,1.495,0.001,0.113,4.457,0.359,0.0,223.022
alerts ~ awareness,1.457,0.0,0.216,4.293,0.347,0.0,368.135
family_communication ~ awareness,1.516,0.0,0.141,4.553,0.365,0.0,259.117


## What is the effect of insurance on preparedness?

In [19]:
determinant = 'insured'
insurance = [
    'made_safer ~ insured',
    'documents ~ insured',
    'learned_routes ~ insured',
    'supplies ~ insured',
    'involved ~ insured',
    'made_plan ~ insured', 
    'practiced_drills ~ insured', 
    'alerts ~ insured', 
    'family_communication ~ insured'
]

In [20]:
probit(functions=insurance, determinant=determinant, data=df_n)

,effect,p,marginal_effect,pseudoR_2,LLRp,BIC
made_safer ~ insured,0.956,0.0,0.231,0.133,0.0,423.62
documents ~ insured,1.252,0.0,0.272,0.209,0.0,383.931
learned_routes ~ insured,1.304,0.0,0.329,0.223,0.0,439.706
supplies ~ insured,0.898,0.0,0.245,0.12,0.0,473.214
involved ~ insured,0.945,0.0,0.137,0.131,0.0,267.118
made_plan ~ insured,0.728,0.0,0.204,0.082,0.0,483.586
practiced_drills ~ insured,0.999,0.0,0.133,0.144,0.0,246.319
alerts ~ insured,1.123,0.0,0.265,0.175,0.0,413.896
family_communication ~ insured,1.071,0.0,0.167,0.162,0.0,283.362


In [21]:
logit(functions=insurance, determinant=determinant, data=df_n)

,effect,p,marginal_effect,odds,pseudoR_2,LLRp,BIC
made_safer ~ insured,1.64,0.0,0.223,5.157,0.312,0.0,423.62
documents ~ insured,2.163,0.0,0.261,8.694,0.441,0.0,383.931
learned_routes ~ insured,2.182,0.0,0.311,8.864,0.445,0.0,439.706
supplies ~ insured,1.507,0.0,0.236,4.513,0.277,0.0,473.214
involved ~ insured,1.81,0.0,0.136,6.113,0.356,0.0,267.118
made_plan ~ insured,1.225,0.0,0.197,3.406,0.202,0.0,483.586
practiced_drills ~ insured,1.946,0.0,0.133,6.998,0.39,0.0,246.319
alerts ~ insured,1.92,0.0,0.254,6.818,0.383,0.0,413.896
family_communication ~ insured,2.01,0.0,0.165,7.462,0.405,0.0,283.362


# Socioeconoimc predictors 

## What is the effect of age on preparedness?

In [25]:
determinant = 'age'
age = [
    'made_safer ~ age',
    'documents ~ age',
    'insured ~ age',
    'learned_routes ~ age',
    'supplies ~ age',
    'involved ~ age',
    'made_plan ~ age', 
    'practiced_drills ~ age', 
    'alerts ~ age', 
    'family_communication ~ age'
]

In [26]:
probit(functions=age, determinant=determinant, data=df_n)

,effect,p,marginal_effect,pseudoR_2,LLRp,BIC
made_safer ~ age,-0.003,0.341,-0.001,0.004,0.34,462.527
documents ~ age,0.004,0.29,0.001,0.005,0.29,450.191
insured ~ age,-0.006,0.071,-0.002,0.014,0.069,499.124
learned_routes ~ age,-0.0,0.931,-0.0,0.0,0.931,517.584
supplies ~ age,-0.01,0.003,-0.003,0.038,0.003,501.036
involved ~ age,-0.019,0.0,-0.003,0.127,0.0,278.133
made_plan ~ age,-0.006,0.05,-0.002,0.017,0.048,503.677
practiced_drills ~ age,-0.018,0.0,-0.003,0.113,0.0,261.931
alerts ~ age,0.002,0.454,0.001,0.002,0.454,468.77
family_communication ~ age,-0.01,0.017,-0.002,0.037,0.015,316.822


In [27]:
logit(functions=age, determinant=determinant, data=df_n)

,effect,p,marginal_effect,odds,pseudoR_2,LLRp,BIC
made_safer ~ age,-0.006,0.34,-0.001,0.994,0.013,0.337,462.517
documents ~ age,0.006,0.29,0.001,1.006,0.016,0.291,450.195
insured ~ age,-0.01,0.072,-0.002,0.99,0.041,0.07,499.136
learned_routes ~ age,-0.0,0.931,-0.0,1.0,0.0,0.931,517.584
supplies ~ age,-0.017,0.003,-0.003,0.983,0.105,0.003,501.122
involved ~ age,-0.037,0.0,-0.003,0.964,0.354,0.0,278.742
made_plan ~ age,-0.011,0.052,-0.002,0.989,0.047,0.05,503.727
practiced_drills ~ age,-0.035,0.0,-0.003,0.965,0.334,0.0,262.315
alerts ~ age,0.004,0.454,0.001,1.004,0.008,0.454,468.77
family_communication ~ age,-0.019,0.019,-0.002,0.981,0.128,0.016,316.858


## What is the effect of income on preparedness? 

In [28]:
determinant = 'income'
income = [
    'made_safer ~ income',
    'documents ~ income',
    'insured ~ income',
    'learned_routes ~ income',
    'supplies ~ income',
    'involved ~ income',
    'made_plan ~ income', 
    'practiced_drills ~ income', 
    'alerts ~ income', 
    'family_communication ~ income'
]

In [29]:
probit(functions=income, determinant=determinant, data=df_n)

,effect,p,marginal_effect,pseudoR_2,LLRp,BIC
made_safer ~ income,-0.0,0.112,-0.0,0.013,0.104,460.799
documents ~ income,0.0,0.759,0.0,0.0,0.759,451.216
insured ~ income,0.0,0.561,0.0,0.001,0.562,502.09
learned_routes ~ income,0.0,0.978,0.0,0.0,0.978,517.591
supplies ~ income,-0.0,0.222,-0.0,0.007,0.216,508.588
involved ~ income,0.0,0.912,0.0,0.0,0.912,295.67
made_plan ~ income,-0.0,0.96,-0.0,0.0,0.96,507.578
practiced_drills ~ income,-0.0,0.601,-0.0,0.002,0.596,276.362
alerts ~ income,-0.0,0.862,-0.0,0.0,0.861,469.3
family_communication ~ income,0.0,0.927,0.0,0.0,0.927,322.684


In [30]:
logit(functions=income, determinant=determinant, data=df_n)

,effect,p,marginal_effect,odds,pseudoR_2,LLRp,BIC
made_safer ~ income,-0.0,0.121,-0.0,1.0,0.041,0.107,460.837
documents ~ income,0.0,0.761,0.0,1.0,0.001,0.762,451.218
insured ~ income,0.0,0.558,0.0,1.0,0.004,0.561,502.088
learned_routes ~ income,0.0,0.978,0.0,1.0,0.0,0.978,517.591
supplies ~ income,-0.0,0.23,-0.0,1.0,0.02,0.219,508.606
involved ~ income,0.0,0.912,0.0,1.0,0.0,0.913,295.671
made_plan ~ income,-0.0,0.961,-0.0,1.0,0.0,0.961,507.578
practiced_drills ~ income,-0.0,0.608,-0.0,1.0,0.008,0.601,276.37
alerts ~ income,-0.0,0.865,-0.0,1.0,0.0,0.864,469.301
family_communication ~ income,0.0,0.928,0.0,1.0,0.0,0.928,322.685


## What is the effect of gender on preparedness?

In [31]:
determinant = 'sex'
sex = [
     'made_safer ~ sex',
    'documents ~ sex',
    'insured ~ sex',
    'learned_routes ~ sex',
    'supplies ~ sex',
    'involved ~ sex',
    'made_plan ~ sex', 
    'practiced_drills ~ sex', 
    'alerts ~ sex', 
    'family_communication ~ sex'
]

In [32]:
probit(functions=sex, determinant=determinant, data=df_n)

,effect,p,marginal_effect,pseudoR_2,LLRp,BIC
made_safer ~ sex,0.094,0.484,0.025,0.002,0.484,462.947
documents ~ sex,0.065,0.633,0.017,0.001,0.633,451.082
insured ~ sex,0.083,0.521,0.024,0.002,0.521,502.014
learned_routes ~ sex,0.136,0.289,0.041,0.005,0.289,516.467
supplies ~ sex,0.096,0.458,0.028,0.002,0.458,509.569
involved ~ sex,0.039,0.813,0.006,0.0,0.813,295.626
made_plan ~ sex,0.053,0.681,0.016,0.001,0.681,507.411
practiced_drills ~ sex,-0.014,0.935,-0.002,0.0,0.935,276.636
alerts ~ sex,0.251,0.062,0.067,0.016,0.061,465.823
family_communication ~ sex,0.178,0.261,0.031,0.008,0.259,321.42


In [33]:
logit(functions=sex, determinant=determinant, data=df_n)

,effect,p,marginal_effect,odds,pseudoR_2,LLRp,BIC
made_safer ~ sex,0.166,0.484,0.025,1.181,0.007,0.484,462.947
documents ~ sex,0.115,0.633,0.017,1.122,0.003,0.633,451.082
insured ~ sex,0.144,0.521,0.024,1.155,0.005,0.521,502.014
learned_routes ~ sex,0.234,0.29,0.041,1.263,0.014,0.289,516.467
supplies ~ sex,0.165,0.459,0.028,1.179,0.007,0.458,509.569
involved ~ sex,0.077,0.813,0.006,1.08,0.001,0.813,295.626
made_plan ~ sex,0.092,0.681,0.016,1.096,0.002,0.681,507.411
practiced_drills ~ sex,-0.028,0.935,-0.002,0.973,0.0,0.935,276.636
alerts ~ sex,0.444,0.063,0.067,1.559,0.047,0.061,465.823
family_communication ~ sex,0.346,0.262,0.032,1.413,0.029,0.259,321.42


## What is the effect of education on preparedness? 

In [34]:
determinant = 'education'
education = [
    'made_safer ~ education',
    'documents ~ education',
    'insured ~ education',
    'learned_routes ~ education',
    'supplies ~ education',
    'involved ~ education',
    'made_plan ~ education', 
    'practiced_drills ~ education', 
    'alerts ~ education', 
    'family_communication ~ education'
]

In [35]:
probit(functions=education, determinant=determinant, data=df_n)

,effect,p,marginal_effect,pseudoR_2,LLRp,BIC
made_safer ~ education,-0.057,0.222,-0.015,0.007,0.221,461.94
documents ~ education,0.023,0.635,0.006,0.001,0.635,451.084
insured ~ education,0.06,0.181,0.017,0.008,0.181,500.633
learned_routes ~ education,-0.009,0.845,-0.003,0.0,0.845,517.554
supplies ~ education,-0.063,0.154,-0.019,0.008,0.153,508.082
involved ~ education,0.039,0.477,0.006,0.003,0.476,295.175
made_plan ~ education,-0.045,0.315,-0.013,0.004,0.314,506.567
practiced_drills ~ education,0.007,0.908,0.001,0.0,0.908,276.63
alerts ~ education,0.018,0.695,0.005,0.001,0.695,469.176
family_communication ~ education,0.055,0.309,0.01,0.006,0.308,321.655


In [36]:
logit(functions=education, determinant=determinant, data=df_n)

,effect,p,marginal_effect,odds,pseudoR_2,LLRp,BIC
made_safer ~ education,-0.101,0.221,-0.015,0.904,0.021,0.22,461.931
documents ~ education,0.039,0.64,0.006,1.04,0.003,0.64,451.091
insured ~ education,0.104,0.182,0.017,1.109,0.022,0.181,500.638
learned_routes ~ education,-0.015,0.846,-0.003,0.985,0.0,0.846,517.554
supplies ~ education,-0.111,0.153,-0.019,0.895,0.025,0.151,508.06
involved ~ education,0.081,0.469,0.007,1.084,0.014,0.469,295.158
made_plan ~ education,-0.078,0.316,-0.013,0.925,0.012,0.315,506.571
practiced_drills ~ education,0.014,0.906,0.001,1.014,0.0,0.906,276.629
alerts ~ education,0.032,0.696,0.005,1.032,0.002,0.696,469.177
family_communication ~ education,0.108,0.309,0.01,1.114,0.024,0.307,321.651


## What is the effect of homeownership on preparedness?

In [37]:
determinant = 'homeownership'
homeownership = [
    'made_safer ~ homeownership',
    'documents ~ homeownership',
    'insured ~ homeownership',
    'learned_routes ~ homeownership',
    'supplies ~ homeownership',
    'involved ~ homeownership',
    'made_plan ~ homeownership', 
    'practiced_drills ~ homeownership', 
    'alerts ~ homeownership', 
    'family_communication ~ homeownership'
]

In [38]:
probit(functions=homeownership, determinant=determinant, data=df_n)

,effect,p,marginal_effect,pseudoR_2,LLRp,BIC
made_safer ~ homeownership,0.153,0.26,0.041,0.006,0.259,462.162
documents ~ homeownership,0.064,0.639,0.017,0.001,0.639,451.089
insured ~ homeownership,0.33,0.013,0.095,0.026,0.013,496.21
learned_routes ~ homeownership,-0.068,0.597,-0.02,0.001,0.597,517.312
supplies ~ homeownership,0.037,0.776,0.011,0.0,0.776,510.039
involved ~ homeownership,-0.39,0.018,-0.062,0.036,0.018,290.048
made_plan ~ homeownership,-0.063,0.626,-0.019,0.001,0.626,507.343
practiced_drills ~ homeownership,-0.063,0.71,-0.009,0.001,0.71,276.505
alerts ~ homeownership,0.083,0.538,0.022,0.002,0.538,468.95
family_communication ~ homeownership,-0.031,0.844,-0.006,0.0,0.844,322.654


In [39]:
logit(functions=homeownership, determinant=determinant, data=df_n)

,effect,p,marginal_effect,odds,pseudoR_2,LLRp,BIC
made_safer ~ homeownership,0.272,0.262,0.041,1.313,0.018,0.259,462.162
documents ~ homeownership,0.115,0.639,0.017,1.121,0.003,0.639,451.089
insured ~ homeownership,0.576,0.014,0.096,1.779,0.076,0.013,496.21
learned_routes ~ homeownership,-0.117,0.596,-0.02,0.89,0.003,0.597,517.312
supplies ~ homeownership,0.064,0.776,0.011,1.066,0.001,0.776,510.039
involved ~ homeownership,-0.776,0.02,-0.062,0.46,0.13,0.018,290.048
made_plan ~ homeownership,-0.109,0.626,-0.019,0.897,0.003,0.626,507.343
practiced_drills ~ homeownership,-0.126,0.71,-0.009,0.882,0.004,0.71,276.505
alerts ~ homeownership,0.147,0.539,0.022,1.158,0.005,0.538,468.95
family_communication ~ homeownership,-0.06,0.844,-0.006,0.941,0.001,0.844,322.654


## What is the effect of monthly rent or mortgage payments on preparedness? 

In [40]:
determinant = 'rentmortgage'
rentmortgage = [
    'made_safer ~ rentmortgage',
    'documents ~ rentmortgage',
    'insured ~ rentmortgage',
    'learned_routes ~ rentmortgage',
    'supplies ~ rentmortgage',
    'involved ~ rentmortgage',
    'made_plan ~ rentmortgage', 
    'practiced_drills ~ rentmortgage', 
    'alerts ~ rentmortgage', 
    'family_communication ~ rentmortgage'
]

In [41]:
probit(functions=rentmortgage, determinant=determinant, data=df_n)

,effect,p,marginal_effect,pseudoR_2,LLRp,BIC
made_safer ~ rentmortgage,0.0,0.374,0.0,0.004,0.374,462.647
documents ~ rentmortgage,0.0,0.664,0.0,0.001,0.664,451.122
insured ~ rentmortgage,0.0,0.157,0.0,0.008,0.157,500.423
learned_routes ~ rentmortgage,0.0,0.293,0.0,0.005,0.293,516.486
supplies ~ rentmortgage,0.0,0.092,0.0,0.012,0.092,507.282
involved ~ rentmortgage,0.0,0.251,0.0,0.009,0.252,294.368
made_plan ~ rentmortgage,0.0,0.096,0.0,0.011,0.096,504.801
practiced_drills ~ rentmortgage,0.0,0.005,0.0,0.053,0.005,268.619
alerts ~ rentmortgage,0.0,0.483,0.0,0.002,0.484,468.84
family_communication ~ rentmortgage,0.0,0.019,0.0,0.033,0.018,317.139


In [42]:
logit(functions=rentmortgage, determinant=determinant, data=df_n)

,effect,p,marginal_effect,odds,pseudoR_2,LLRp,BIC
made_safer ~ rentmortgage,0.0,0.372,0.0,1.0,0.011,0.373,462.644
documents ~ rentmortgage,0.0,0.664,0.0,1.0,0.003,0.665,451.122
insured ~ rentmortgage,0.0,0.161,0.0,1.0,0.024,0.162,500.466
learned_routes ~ rentmortgage,0.0,0.295,0.0,1.0,0.013,0.296,516.499
supplies ~ rentmortgage,0.0,0.093,0.0,1.0,0.033,0.094,507.307
involved ~ rentmortgage,0.0,0.258,0.0,1.0,0.031,0.26,294.412
made_plan ~ rentmortgage,0.0,0.095,0.0,1.0,0.033,0.095,504.795
practiced_drills ~ rentmortgage,0.001,0.005,0.0,1.001,0.181,0.005,268.688
alerts ~ rentmortgage,0.0,0.486,0.0,1.0,0.007,0.487,468.848
family_communication ~ rentmortgage,0.0,0.019,0.0,1.0,0.11,0.019,317.184


# Safe results 

In [ ]:
# probit
functions = [
    ('perception', perception, df_k),
    ('awareness', awareness, df_k),
    ('experience', experience, df_n),
    ('floodzone', floodzone, df_k),
    ('insured', insurance, df_n),
    ('age', age, df_n),
    ('income', income, df_n),
    ('sex', sex, df_n),
    ('education', education, df_n),
    ('homeownership', homeownership, df_n),
    ('rentmortgage', rentmortgage, df_n)
]



# probit results 
res_list = [] 
determinant_keys = [] # first level index 

for determinant, formula_list, data in functions:
    res_df = probit(functions=formula_list, determinant=determinant, data=data)
    res_list.append(res_df)
    determinant_keys.append(determinant)
probit_df = pd.concat(res_list, keys = determinant_keys, names = ['Determinant','Function'])
probit_df.to_excel('results/probit_univariate.xlsx')

#logit results 
res_llist = []
determinant_keys = []

for determinant, formula_list, data in functions: 
    res_df = logit(functions=formula_list, determinant=determinant, data=data)
    res_list.append(res_df)
    determinant_keys.append(determinant)
logit_df = pd.concat(res_list, keys = determinant_keys, names = ['Determinant', 'Function'])
logit_df.to_excel('results/logit_univariate.xlsx')


## What is the effect of risk perception on awareness?

In [44]:
functions = ['awareness ~ perception']
determinant = 'perception'

In [50]:
probit(functions=functions, determinant='perception', data=df_k)

,effect,p,marginal_effect,pseudoR_2,LLRp,BIC
awareness ~ perception,0.71,0.0,0.266,0.111,0.0,515.788


In [51]:
functinos = ['perception ~ awareness']
determinant = 'awareness'

In [52]:
probit(functions=functinos, determinant='awareness', data=df_k)

,effect,p,marginal_effect,pseudoR_2,LLRp,BIC
perception ~ awareness,0.708,0.0,0.262,0.111,0.0,509.974
